In [1]:
import sys
from qwikidata.sparql import return_sparql_query_results
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from SPARQLWrapper import SPARQLWrapper, JSON
import json
from tqdm import tqdm
import re
import requests as r
import pandas as pd
from collections import defaultdict
# from libindic import inexactsearch
import pandas as pd
import pickle as pkl
import wptools
sys.path.append('..')
from method2.People_translator import People_translator
translator = People_translator()
from method1.utils import *
from baseline.baseline import baseline_infobox
from method1.get_infobox import method1_infobox

Syntax
For method1 -> method1_infobox(qid , category)
    qid - Qid of the wikidata item
    category - 'people'/'places'

For method2 -> translator.get_infobox(Name , test)
    translator - instancse of People_Translator / Places_Translator
    Name       - Name of the page in the English Wikipedia
    test       - True / False ( default = False ) to not pretty print the generated infobox
    
For baseline -> baseline_infobox(qid , category)
    qid - Qid of the wikidata item
    category - 'people'/'places'

In [2]:
def people_format(infobox):
    infobox = [ ("|" + key + " = " +  val) for key , val in infobox.items()]
    print("{{Infobox person")
    print("\n".join(infobox))
    print("}}")
    
def place_format(infobox):
    infobox = [ ("|" + key + " = " +  val) for key , val in infobox.items()]
    print("{{Infobox place")
    print("\n".join(infobox))
    print("}}")

In [3]:
m1_infobox = method1_infobox("Q213854" , 'people')

In [4]:
m2_infobox = translator.get_infobox("Virat Kohli" , test=True)

en.wikipedia.org (parse) Virat Kohli
en.wikipedia.org (imageinfo) File:The President, Shri Pranab Mukh...
Virat Kohli (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:The Presi...
  infobox: <dict(97)> name, image, caption, birth_date, birth_plac...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:V...
  pageid: 16017429
  parsetree: <str(264279)> <root><template><title>short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Virat Kohli
  wikibase: Q213854
  wikidata_url: https://www.wikidata.org/wiki/Q213854
  wikitext: <str(222289)> {{short description|Indian international...
}


In [5]:
b_infobox = baseline_infobox("Q213854",category='people')

### Method 1 Infobox

In [6]:
people_format(m1_infobox)

{{Infobox person
|name = विराट कोहली
|image = http://commons.wikimedia.org/wiki/Special:FilePath/Virat%20Kohli%20in%20New%20Delhi%20on%20December%202018.jpghttp://commons.wikimedia.org/wiki/Special:FilePath/Virat%20Kohli%20portrait.jpghttp://commons.wikimedia.org/wiki/Special:FilePath/The%20President%2C%20Shri%20Pranab%20Mukherjee%20presenting%20the%20Padma%20Shri%20Award%20to%20Shri%20Virat%20Kohli%2C%20at%20a%20Civil%20Investiture%20Ceremony%2C%20at%20Rashtrapati%20Bhavan%2C%20in%20New%20Delhi%20on%20March%2030%2C%202017%20%28cropped%29.jpg
|gender = पुरुष
|place_of_birth = दिल्ली
|date_of_birth = 1988-11-05T00:00:00Z
|profession = क्रिकेटर
|awards = अर्जुन पुरस्कारराजीव गांधी खेल रत्न
|spouse = अनुष्का शर्मा
|nationality = भारत
}}


### Method 2 Infobox

In [7]:
people_format(m2_infobox)

{{Infobox person
|name = विराट कोहली
|image = The President, Shri Pranab Mukherjee presenting the Padma Shri Award to Shri Virat Kohli, at a Civil Investiture Ceremony, at Rashtrapati Bhavan, in New Delhi on March 30, 2017 (cropped).jpg
|caption = 2017 में कोहली
|nickname = चीकू ऑर चीकू
|birth_date = {{birth date and age|1988|11|5|df=|y}}
|birth_place = [[नई दिल्ली]], भारत
|country = भारत
|family = {{marriage|[[Anushka Sharma]] (wife)|2017}}
|height = 1.75 म
|batting = राइट-हैंडेड
|bowling = राइट-अर्म [[फास्ट बॉलिंग|मीडियम]]
|role = [[बैटिंग ऑडर (क्रिकेट)#टॉप ऑडर|टॉप-ऑडर बैटसमैन]]
|international = ट्रू
|internationalspan = 2008–प्रेज़ेंट
|testdebutdate = 20 जून
}}


### Baseline Infobox

In [8]:
people_format(b_infobox)

{{Infobox person
|name = विराट कोहली
|description = भारतीय क्रिकेट कप्तान
|image = http://commons.wikimedia.org/wiki/Special:FilePath/Virat%20Kohli%20in%20New%20Delhi%20on%20December%202018.jpg,http://commons.wikimedia.org/wiki/Special:FilePath/Virat%20Kohli%20portrait.jpg,http://commons.wikimedia.org/wiki/Special:FilePath/The%20President%2C%20Shri%20Pranab%20Mukherjee%20presenting%20the%20Padma%20Shri%20Award%20to%20Shri%20Virat%20Kohli%2C%20at%20a%20Civil%20Investiture%20Ceremony%2C%20at%20Rashtrapati%20Bhavan%2C%20in%20New%20Delhi%20on%20March%2030%2C%202017%20%28cropped%29.jpg,
|gender = पुरुष
|birth_place = डेल्ही,
|birth_date = 1988-11-05T00:00:00Z,
|profession = क्रिकेट खिलाड़ी
|awards = अर्जुन पुरस्कार,राजीव गांधी खेल रत्न, खेल में पद्मश्री
|spouse = अनुष्का शर्मा,
|nationality = भारत
}}


In [9]:
from method2.Places_translator import Places_translator
translator = Places_translator()

In [10]:
m1_infobox = method1_infobox("Q1353" , category='places')
m2_infobox = translator.get_infobox("Delhi" , test=True)
b_infobox = baseline_infobox("Q1353" , category = 'places')

en.wikipedia.org (parse) Delhi
Delhi (en) data
{
  infobox: <dict(81)> name, official_name, image_blank_emblem, bla...
  iwlinks: <list(9)> https://commons.wikimedia.org/wiki/Delhi_-_%E...
  pageid: 37756
  parsetree: <str(263292)> <root><template><title>short descriptio...
  requests: <list(1)> parse
  title: Delhi
  wikibase: Q1353
  wikidata_url: https://www.wikidata.org/wiki/Q1353
  wikitext: <str(201851)> {{short description|Megacity and union t...
}


In [11]:
place_format(m1_infobox)

{{Infobox place
|name = दिल्ली
|image = http://commons.wikimedia.org/wiki/Special:FilePath/Edificios%20ministeriales%20Delhi.JPG
|map = http://commons.wikimedia.org/wiki/Special:FilePath/National%20Capital%20Territory%20of%20Delhi%20in%20India%20%28special%20marker%29%20%28claimed%20and%20disputed%20hatched%29.svg
|country = भारत
|area / area_km2 = 1484
|population = 26495000
|elevation / elevation_m = 200
}}


In [12]:
place_format(m2_infobox)

{{Infobox place
|name = डेल्ही
|category = [[केंद्र शासित प्रदेश]] और [[मेगासिटी]]
|image = Delhi Montage.jpg
|image_caption = ऊपर से दक्षिणावर्त: [[कमल मंदिर]], [[हुमायूं का मकबरा]], [[कनॉट प्लेस, नई दिल्ली। कनॉट प्लेस]], [[अक्षरधाम (दिल्ली)। अक्षरधाम मंदिर]] और [[इंडिया गेट]]
|map_caption = भारत में दिल्ली का स्थान
|timezone = [[Indian Standard Time|IST]]
|elevation = 200–250
|official_name = नेशनल कैपिटल टेरिटरी ऑफ डेल्ही
|image_blank_emblem = सील_ऑफ_दी_नेशनल_कैपिटल_टेरिटरी_ऑफ_डेल्ही.सवग
|blank_emblem_type = एम्ब्लेम
|settlement_type = [[यूनियन टेरिटरी]] अंड [[मेगासिटी]]
|image_skyline = डेल्ही मोंटेज.जपग
|image_size = 250
|image_map = इन-डल.सवग
|coordinates = {{coord|28|36|36|N|77|13|48|E|display|=|inline, title}}
}}


In [13]:
place_format(b_infobox)

{{Infobox place
|name = दिल्ली
|description = दिल्ली की राजधानी क्या है
|image = http://commons.wikimedia.org/wiki/Special:FilePath/Edificios%20ministeriales%20Delhi.JPG,
|map = http://commons.wikimedia.org/wiki/Special:FilePath/National%20Capital%20Territory%20of%20Delhi%20in%20India%20%28special%20marker%29%20%28claimed%20and%20disputed%20hatched%29.svg,
|country = भारत
|area / area_km2 = 1484,
|population = 26495000,
|elevation / elevation_m = 200,
}}
